In [1]:
# Import the data set

import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
import random


In [4]:

test_file = '../customer_churn_dataset-testing-master.csv'
train_file = '../customer_churn_dataset-training-master.csv'

test_df = pd.read_csv(test_file)
train_df = pd.read_csv(train_file)


df = pd.concat([train_df, test_df], axis=0)
df.reset_index(drop=True, inplace=True)

'''
df = df.sample(n=50000, random_state=42)

'''

'\ndf = df.sample(n=50000, random_state=42)\n\n'

In [6]:

def evaluate_classification(y_true, y_pred):

    auroc = roc_auc_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)

    print("AUROC:", auroc)
    print("F1-Score:", f1)
    print("Precision:", precision)
    print("Recall:", recall)
    print("Accuracy:", accuracy)

## Data preprocessing

In [7]:
# Drop missing values 

df = df.dropna()

In [9]:
# Z Score normalization for Last Interaction 

normalized_last_interaction = (df['Last Interaction'] - df['Last Interaction'].mean()) / df['Last Interaction'].std()

# Replace the previous Last Interaction with new one 
df['Last Interaction'] = normalized_last_interaction

In [10]:
# Create 5 bins for attribute Total Spend 

bin_labels = ['Bin 1', 'Bin 2', 'Bin 3', 'Bin 4', 'Bin 5']

df['Total Spend'] = pd.cut(df['Total Spend'], bins=5, labels=bin_labels)

In [11]:
df.head()

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,Bin 5,0.277572,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,Bin 3,-1.000267,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,Bin 1,-1.348768,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,Bin 2,1.671578,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,Bin 3,0.626073,1.0


In [12]:
# Drop unecessary columns 

df.drop('CustomerID', axis=1, inplace=True)

# Drop gender to ensure equality 
df.drop('Gender', axis=1, inplace=True)

In [13]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, gain=0, y=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
        self.gain = gain
        self.y = y
        self.count = Counter(y)

    def is_leaf_node(self):
        return self.value is not None


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, depth=None, n_features=None, criterion="GINI"):
        self.labels = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None
        self.depth = depth if depth else 0
        self.criterion = criterion

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.labels = X.columns
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        if depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value, y=y)

        feat_idxs = np.random.choice(n_features, self.n_features, replace=False)
        best_feature, best_threshold, best_gain = self._best_split(X, y, feat_idxs)

        left_idxs, right_idxs = self._split(X.iloc[:, best_feature], best_threshold)
        left = self._grow_tree(X.iloc[left_idxs, :], y.iloc[left_idxs], depth + 1)
        right = self._grow_tree(X.iloc[right_idxs, :], y.iloc[right_idxs], depth + 1)
        return Node(best_feature, best_threshold, left, right, best_gain, y)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        best_split_idx, best_split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X.iloc[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                if self.criterion == "GINI":
                    gain = self._gini_gain(y, X_column, thr)
                elif self.criterion == "GAIN_RATIO":
                    gain = self._gain_ratio(y, X_column, thr)
                else:
                    gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    best_split_idx = feat_idx
                    best_split_threshold = thr

        return best_split_idx, best_split_threshold, best_gain

    def _gini_gain(self, y, X_column, threshold):
        gini_base = self._get_gini(y)

        left_idxs, right_idxs = self._split(X_column, threshold)
        left_counts = Counter(y.iloc[left_idxs])
        right_counts = Counter(y.iloc[right_idxs])

        left_class0_count = Counter(left_counts).get(0, 0)
        left_class1_count = Counter(left_counts).get(1, 0)
        right_class0_count = Counter(right_counts).get(0, 0)
        right_class1_count = Counter(right_counts).get(1, 0)

        gini_left = self._gini_impurity(left_class0_count, left_class1_count)
        gini_right = self._gini_impurity(right_class0_count, right_class1_count)

        n_left = left_class0_count + left_class1_count
        n_right = right_class0_count + right_class1_count

        w_left = n_left / (n_left + n_right)
        w_right = n_right / (n_left + n_right)

        w_gini = w_left * gini_left + w_right * gini_right

        gini_gain = gini_base - w_gini
        return gini_gain

    def _get_gini(self, y):
        class0_count = Counter(y).get(0, 0)
        class1_count = Counter(y).get(1, 0)
        return self._gini_impurity(class0_count, class1_count)

    def _gini_impurity(self, class0_count, class1_count):
        if class0_count is None:
            class0_count = 0
        if class1_count is None:
            class1_count = 0
        n = class0_count + class1_count

        if n == 0:
            return 0.0

        p0 = class0_count / n
        p1 = class1_count / n
        gini = 1 - (p0 ** 2 + p1 ** 2)
        return gini

    def _gain_ratio(self, y, X_column, threshold):
        parent_entropy = self._entropy(y)
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        e_left, e_right = self._entropy(y.iloc[left_idxs]), self._entropy(y.iloc[right_idxs])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right

        information_gain = parent_entropy - child_entropy

        split_info = -(n_left / n * np.log(n_left / n) + n_right / n * np.log(n_right / n))

        return information_gain / split_info

    def _information_gain(self, y, X_column, threshold):
        parent_entropy = self._entropy(y)
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        e_left, e_right = self._entropy(y.iloc[left_idxs]), self._entropy(y.iloc[right_idxs])
        child_entropy = (n_left / n) * e_left + (n_right / n) * e_right

        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_threshold):
        left_idxs = np.argwhere(X_column.to_numpy() <= split_threshold).flatten()
        right_idxs = np.argwhere(X_column.to_numpy() > split_threshold).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        counter = Counter(y)
        if len(Counter(y).most_common(1)) == 0:
            value = None
        else:
            value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for idx, x in X.iterrows()])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node() or node.feature is None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

    def counter_to_str(self, counter: Counter):
        inside = [f"{key}: {value}" for key, value in counter.items()]
        return f"({', '.join(inside)})"

    def print_info(self, node, depth):
        if node.feature is None:
            return

        preamble0 = ' ' * depth * 2 + ('\t\t' if depth > 0 else '')

        print(f"{preamble0} {self.labels[node.feature]} <= {node.threshold} ? {node.gain}")

        self.print_info(node.left, depth + 1)
        self.print_info(node.right, depth + 1)

    def print_tree(self):
        self.print_info(self.root, 0)


In [15]:
from sklearn.model_selection import train_test_split


# Splitting into features (X) and target (y)
X = df.drop("Churn", axis=1)
y = df["Churn"]

# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Random forest based on DT

### Method used to create this random forest <br>
<br>

Random forest is advanced version tree, which includes multiple decision trees. This apply the same logic, which I will create multiple decision tree, train them and make prediction for each of the decision tree. And then, the results are aggregated, and then get the mean to get the final prediction. <br> 

The random forest is better because it will be roburst to noise and outliers. 

### Initialize Random Forest 

In [18]:
class RandomForest:
    def __init__(self, n_estimators=100, max_depth=12, criterion="GAIN_RATIO", min_samples_split=2, n_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.criterion = criterion
        self.min_samples_split = min_samples_split
        self.n_features = n_features
        self.estimators = []

    def fit(self, X, y):
        for _ in range(self.n_estimators):
            subset_indices = np.random.choice(len(X), len(X), replace=True)
            subset_X, subset_y = X.iloc[subset_indices], y.iloc[subset_indices]

            tree = DecisionTree(max_depth=self.max_depth, criterion=self.criterion,
                                min_samples_split=self.min_samples_split, n_features=self.n_features)
            tree.fit(subset_X, subset_y)
            self.estimators.append(tree)

    def predict(self, X):
        predictions = np.array([tree.predict(X) for tree in self.estimators])
        return np.round(np.mean(predictions, axis=0))


### Train and evaluate <strong>Random Forest</strong>

In [19]:
# Instantiate and train the RandomForest
rf = RandomForest(n_estimators=85, max_depth=12, criterion="GAIN_RATIO", min_samples_split=2, n_features=None)
rf.fit(X_train, y_train)

# Make predictions using the RandomForest
predictions_rf = rf.predict(X_test)

# EST: 11m for 100 DT

In [20]:
# Check accuracy of the random forest

evaluate_classification(y_test, predictions_rf)

AUROC: 0.9177499262088125
F1-Score: 0.9345289947903322
Precision: 0.8964918317521179
Recall: 0.9759369258486291
Accuracy: 0.924130559569288


### Train and evaluate <strong>decision tree</strong>

In [16]:
# Initialize a decision tree - Gain_Ratio
clf_gain = DecisionTree(max_depth=11, criterion="GAIN_RATIO")
clf_gain.fit(X_train, y_train)

predictions_gain_ratio = clf_gain.predict(X_test)


In [17]:
evaluate_classification(y_test, predictions_gain_ratio)

AUROC: 0.9166877959349955
F1-Score: 0.9333629839971946
Precision: 0.8965919547784935
Recall: 0.9732791066873584
Accuracy: 0.9228934502484115


## Comparision

Based on the comprehensive evaluation metrics, it is evident that the Random Forest outperforms the Decision Tree across various measurements, including AUROC, F1-Score, precision, recall, and accuracy. This improvement highlights the ensemble nature of Random Forest, which leverages multiple decision trees to enhance predictive performance. However, it's important to note that this custom Random Forest implementation, while exhibiting notable advancements, may not be as optimized as established libraries like scikit-learn's Random Forest, due to being developed from scratch.